In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, to_hetero
import pandas as pd
import numpy as np
import networkx as nx
import community as community_louvain # Pour l'algorithme de Louvain
import geopandas as gpd
from shapely.geometry import Point 
from shapely.ops import transform 
from sklearn.preprocessing import MinMaxScaler

In [2]:
from src.utils import telecharger_donnees_plu, load_and_prepare_real_data, perform_semantic_sjoin
from src.hetero import HeteroGNN

In [19]:
doc_urba = gpd.read_file("data/wfs_du.gpkg",layer='zone_urba')

In [20]:
doc_urba = doc_urba[['gid','partition','libelle','libelong','typezone','geometry']].copy()

In [23]:
doc_urba.crs ='4326'

In [25]:
doc_urba = doc_urba.to_crs(2154)

In [29]:
df_bat = gpd.read_file('data/BDT_3-5_GPKG_LAMB93_D092-ED2025-06-15.gpkg', layer='batiment')

In [30]:
df_parcelles = gpd.read_file('data/cadastre-92-parcelles.json')

In [8]:
# ept_92 = ['200057990', '200057974', '200057982','200057966', '92026', '92033','92035','92044','92050','92051','92062','92063','92064','92073','92076']
# gdf_usage_sol_92 = gpd.GeoDataFrame()
# for CODE_INSEE in ept_92:
#     gdf_usage_sol = telecharger_donnees_plu(CODE_INSEE)
#     if gdf_usage_sol is not None and not gdf_usage_sol.empty:
#         gdf_usage_sol = gdf_usage_sol[['gid','partition','libelle','libelong','typezone','geometry']].copy()
#         gdf_usage_sol_92 = pd.concat([gdf_usage_sol_92,gdf_usage_sol],axis=0)

Succès de la requête, mais 0 zones retournées. 200057982 n'est peut-être couverte par un PLU/PLUI
Succès de la requête, mais 0 zones retournées. 92033 n'est peut-être couverte par un PLU/PLUI


In [9]:
df_parcelles['superficie'] = df_parcelles.to_crs(2154).area

In [29]:
gdf_par = perform_semantic_sjoin(df_parcelles, gdf_usage_sol_92)

Reprojection des Parcelles vers EPSG:2154...
Reprojection des Zones PLU vers EPSG:2154...
Calcul du point représentatif des Parcelles (point garanti dans la géométrie)...
Exécution de la jointure spatiale (Parcelle.ReprensentativePoint WITHIN Zone PLU)...


In [32]:
gdf_par = gpd.GeoDataFrame(
        gdf_par, 
        geometry=gdf_par['geometry'], 
        crs="EPSG:2154"
    )

In [33]:
gdf_bat = df_bat[['cleabs','hauteur','geometry']].copy()

In [34]:
gdf_bat['surface'] = gdf_bat.force_2d().area

In [35]:
gdf_bat['geometry'] = gdf_bat['geometry'].force_2d()

In [32]:
df_bat

,cleabs,nature,usage_1,usage_2,construction_legere,etat_de_l_objet,date_creation,date_modification,date_d_apparition,date_de_confirmation,...,materiaux_de_la_toiture,hauteur,altitude_minimale_sol,altitude_minimale_toit,altitude_maximale_toit,altitude_maximale_sol,origine_du_batiment,appariement_fichiers_fonciers,identifiants_rnb,geometry
0,BATIMENT0000000329826060,Indifférenciée,Commercial et services,None,False,En service,2013-11-22 14:36:46.100,2018-12-03 21:01:36.714,None,NaT,...,None,4.8,29.0,33.8,NaN,NaN,Cadastre,None,CGRDXG5M86K6,"MULTIPOLYGON Z (((640242.8 6865948.5 33.8, 640..."
1,BATIMENT0000000329826061,Indifférenciée,Commercial et services,None,False,En service,2013-11-22 14:36:46.100,2018-12-03 21:01:36.714,None,NaT,...,None,4.8,29.0,33.8,NaN,NaN,Cadastre,None,M2A1DXRFC24E,"MULTIPOLYGON Z (((640105.5 6865894.8 33.8, 640..."
2,BATIMENT0000000329826062,Indifférenciée,Commercial et services,None,False,En service,2013-11-22 14:36:46.100,2018-12-03 21:01:36.714,None,NaT,...,None,4.8,29.0,33.8,NaN,NaN,Cadastre,None,Y4EGQR8CJABN,"MULTIPOLYGON Z (((640116.9 6865887.7 33.8, 640..."
3,BATIMENT0000000329826063,Indifférenciée,Commercial et services,None,False,En service,2013-11-22 14:36:46.100,2018-12-03 21:01:36.714,None,NaT,...,None,19.3,27.9,NaN,NaN,NaN,Imagerie aérienne,None,6KRE9DN597XW,"MULTIPOLYGON Z (((640125.8 6865891.7 47.2, 640..."
4,BATIMENT0000000329826074,Indifférenciée,Sportif,None,False,En service,2013-11-22 14:44:32.253,2017-12-20 14:55:38.575,None,2013-01-01,...,None,4.5,28.2,NaN,NaN,NaN,Imagerie aérienne,None,MYH2T9PMH7HS,"MULTIPOLYGON Z (((638592.3 6864340.5 32.7, 638..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691546,BATIMENT0000000329742062,Indifférenciée,Commercial et services,None,False,En service,2013-11-20 10:52:27.838,2018-12-03 19:55:36.025,None,NaT,...,None,15.3,98.8,114.1,NaN,NaN,Cadastre,None,XVXAKK7ABTMB,"MULTIPOLYGON Z (((643154.2 6857844 114.1, 6431..."
691547,BATIMENT0000000329742063,Indifférenciée,Commercial et services,None,False,En service,2013-11-20 10:52:27.838,2018-12-03 19:55:36.025,None,NaT,...,None,4.9,98.8,103.7,106.0,99.5,Cadastre,None,3S88EY3HZ2HJ,"MULTIPOLYGON Z (((643128.2 6857845.5 103.7, 64..."
691548,BATIMENT0000000329742064,Indifférenciée,Commercial et services,None,False,En service,2013-11-20 10:52:27.838,2018-12-03 19:55:36.025,None,2016-01-01,...,None,4.9,98.8,103.7,NaN,NaN,Cadastre,None,G5FN44Z5B6NX,"MULTIPOLYGON Z (((643158.7 6857809.7 103.7, 64..."
691549,BATIMENT0000000329742066,Indifférenciée,Commercial et services,None,False,En service,2013-11-20 10:52:30.091,2018-12-03 21:15:12.040,None,NaT,...,None,11.7,36.1,47.8,NaN,NaN,Imagerie aérienne,None,SC93659MPR1A,"MULTIPOLYGON Z (((644106 6861101.3 47.8, 64411..."


In [36]:
df_adr = pd.read_csv('data/adresses-92.csv', delimiter=';')

/tmp/ipykernel_5992/3451560825.py:1: DtypeWarning: Columns (16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_adr = pd.read_csv('data/adresses-92.csv', delimiter=';')


In [37]:
gdf_adr = gpd.GeoDataFrame(
    df_adr[['id','lon','lat','cad_parcelles']], 
    geometry=gpd.points_from_xy(df_adr['lon'], df_adr['lat']), 
    crs="EPSG:4326"
)

In [38]:
sjoin_geom = gpd.sjoin(gdf_bat[['cleabs', 'geometry']].to_crs(2154), gdf_par[['id', 'geometry']].to_crs(2154), how='left', predicate='intersects', lsuffix='bat', rsuffix='par')

In [41]:
sjoin_geom

,cleabs,geometry,index_par,id
0,BATIMENT0000000329826060,"MULTIPOLYGON (((640242.8 6865948.5, 640263 686...",153563.0,92050000CX0305
1,BATIMENT0000000329826061,"MULTIPOLYGON (((640105.5 6865894.8, 640110.7 6...",153563.0,92050000CX0305
2,BATIMENT0000000329826062,"MULTIPOLYGON (((640116.9 6865887.7, 640125.8 6...",153563.0,92050000CX0305
3,BATIMENT0000000329826063,"MULTIPOLYGON (((640125.8 6865891.7, 640142.4 6...",153563.0,92050000CX0305
4,BATIMENT0000000329826074,"MULTIPOLYGON (((638592.3 6864340.5, 638585.6 6...",159742.0,92063000BP0041
...,...,...,...,...
691547,BATIMENT0000000329742063,"MULTIPOLYGON (((643128.2 6857845.5, 643130.9 6...",6405.0,92048000AB0455
691548,BATIMENT0000000329742064,"MULTIPOLYGON (((643158.7 6857809.7, 643161 685...",6405.0,92048000AB0455
691549,BATIMENT0000000329742066,"MULTIPOLYGON (((644106 6861101.3, 644116 68611...",24469.0,920120000J0083
691549,BATIMENT0000000329742066,"MULTIPOLYGON (((644106 6861101.3, 644116 68611...",24531.0,920120000J0115


In [14]:
data = HeteroData()
data['adresse'].x = adr_x
data['bâtiment'].x = bat_x
data['parcelle'].x = par_x

# Relations
data['adresse', 'accès', 'bâtiment'].edge_index = edge_index_ab
data['bâtiment', 'appartient', 'parcelle'].edge_index = edge_index_bp
data['bâtiment', 'appartient', 'parcelle'].edge_attr = edge_attr_bp 

print(f"Structure du Graphe créée. Nœuds: {len(data['bâtiment'].x)} | Arêtes B->P: {data['bâtiment', 'appartient', 'parcelle'].num_edges}")

Structure du Graphe créée. Nœuds: 691551 | Arêtes B->P: 0


In [14]:
print("\n--- 2. Passage de Messages GNN (Création d'Embeddings) ---")
    
HIDDEN_CHANNELS = 64
OUT_CHANNELS = 32

metadata = data.metadata() 
model = HeteroGNN(metadata, hidden_channels=HIDDEN_CHANNELS, out_channels=OUT_CHANNELS)

edge_attr_dict = {
    ('bâtiment', 'appartient', 'parcelle'): data['bâtiment', 'appartient', 'parcelle'].edge_attr
}

x_dict = model(data.x_dict, data.edge_index_dict, edge_attr_dict)

bat_embeddings = x_dict['bâtiment']


--- 2. Passage de Messages GNN (Création d'Embeddings) ---
Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception
